In [1]:
from typing import *
import pandas as pd
from pathlib import Path
import os
import random
import sys
import re
from dotenv import load_dotenv, dotenv_values
from pydriller import Repository
import json
from multiprocessing import Pool
from itertools import chain
from pymongo import MongoClient
import collections

In [2]:
import dotenv
dotenv.load_dotenv()
ENV = dotenv.dotenv_values(".env")
DATA_DIR = Path(ENV["DATA_DIR"])

client = MongoClient("localhost", 42692)
db = client.s5_snyk_libio

In [3]:
db.mergedVuln.drop()

In [4]:
libio_vuln_data = list(db.libioVuln.find())
mvn_eco_vuln_data = list(db.mvnEcoVuln.find())
db.mergedVuln.insert_many(db.libioVuln.find())
db.mergedVuln.insert_many(db.mvnEcoVuln.find())

InsertManyResult([ObjectId('67f9147e6e91fd4e04c23faf'), ObjectId('67f9147e6e91fd4e04c23fb0'), ObjectId('67f9147e6e91fd4e04c23fb1'), ObjectId('67f9147e6e91fd4e04c23fb2'), ObjectId('67f9147e6e91fd4e04c23fb3'), ObjectId('67f9147e6e91fd4e04c23fb4'), ObjectId('67f9147e6e91fd4e04c23fb5'), ObjectId('67f9147e6e91fd4e04c23fb6'), ObjectId('67f9147e6e91fd4e04c23fb7'), ObjectId('67f9147e6e91fd4e04c23fb9'), ObjectId('67f9147e6e91fd4e04c23fba'), ObjectId('67f9147e6e91fd4e04c23fbb'), ObjectId('67f9147e6e91fd4e04c23fbc'), ObjectId('67f9147e6e91fd4e04c23fbd'), ObjectId('67f9147e6e91fd4e04c23fbe'), ObjectId('67f9147e6e91fd4e04c23fbf'), ObjectId('67f9147e6e91fd4e04c23fc0'), ObjectId('67f9147e6e91fd4e04c23fc1'), ObjectId('67f9147e6e91fd4e04c23fc2'), ObjectId('67f9147e6e91fd4e04c23fc3'), ObjectId('67f9147e6e91fd4e04c23fc4'), ObjectId('67f9147e6e91fd4e04c23fc5'), ObjectId('67f9147e6e91fd4e04c23fc6'), ObjectId('67f9147e6e91fd4e04c23fc7'), ObjectId('67f9147e6e91fd4e04c23fc8'), ObjectId('67f9147e6e91fd4e04c23f

In [5]:
len(libio_vuln_data), len(mvn_eco_vuln_data)

(6028, 829)

In [6]:
print(len(set((':'.join(x['vuln_gav'].split(':')[:-1]), x['cve_ref']) for x in libio_vuln_data)))
print(len(set((':'.join(x['vuln_gav'].split(':')[:-1]), x['cve_ref']) for x in mvn_eco_vuln_data)))
print(len(set((':'.join(x['vuln_gav'].split(':')[:-1]), x['cve_ref']) for x in libio_vuln_data + mvn_eco_vuln_data)))

268
829
977


In [7]:
print(len(set((x['vuln_gav'], x['cve_ref']) for x in libio_vuln_data)))  # 6726  some do not have CVE ID
print(len(set((x['vuln_gav'], x['cve_ref']) for x in mvn_eco_vuln_data)))  # 829
print(len(set((x['vuln_gav'], x['cve_ref']) for x in libio_vuln_data + mvn_eco_vuln_data)))  # 7490

6028
829
6787


In [8]:
libio_group_by_gav_cve = collections.defaultdict(list)
for it in libio_vuln_data:
    libio_group_by_gav_cve[(it['vuln_gav'], it['cve_ref'])].append(it)

mvn_eco_group_by_gav_cve = collections.defaultdict(list)
for it in mvn_eco_vuln_data:
    mvn_eco_group_by_gav_cve[(it['vuln_gav'], it['cve_ref'])].append(it)

In [9]:
l = list(sorted(libio_group_by_gav_cve.items(), key=lambda x: len(x[1]), reverse=True))
l[0]

(('ch.qos.logback:logback-classic:1.0.12', 'CVE-2017-5929'),
 [{'_id': ObjectId('67f919b35878e3c56f2efb85'),
   'data_src': 'snyk',
   'cve_ref': 'CVE-2017-5929',
   'snyk_url': 'https://security.snyk.io/vuln/SNYK-JAVA-CHQOSLOGBACK-31407',
   'vuln_gav': 'ch.qos.logback:logback-classic:1.0.12',
   'vuln_classes': ['ch.qos.logback.classic.net.server.RemoteAppenderStreamClient'],
   'patches': ['https://github.com/qos-ch/logback/commit/979b042cb1f0b4c1e5869ccc8912e68c39f769f9']}])

In [10]:
shared_gav_cve_items = list(set(libio_group_by_gav_cve.keys()).intersection(set(mvn_eco_group_by_gav_cve.keys())))
len(shared_gav_cve_items)

70

In [11]:
_rk = random.choice(shared_gav_cve_items)
libio_group_by_gav_cve[_rk], mvn_eco_group_by_gav_cve[_rk]  
# 'CVE-2021-37137', 'io.netty:netty-codec:4.1.5.Final'   mvn_eco data has the wrong cve_ref
# 'CVE-2013-7285' mvn_eco data includes 'ReflectionConverter' which is wrong
# 

([{'_id': ObjectId('67f919b35878e3c56f2efee4'),
   'data_src': 'snyk',
   'cve_ref': 'CVE-2020-7692',
   'snyk_url': 'https://security.snyk.io/vuln/SNYK-JAVA-COMGOOGLEOAUTHCLIENT-575276',
   'vuln_gav': 'com.google.oauth-client:google-oauth-client:1.23.0',
   'vuln_classes': ['com.google.api.client.auth.oauth2.AuthorizationCodeFlow',
    'com.google.api.client.auth.oauth2.AuthorizationCodeRequestUrl'],
   'patches': ['https://github.com/googleapis/google-oauth-java-client/commit/13433cd7dd06267fc261f0b1d4764f8e3432c824']}],
 [{'_id': ObjectId('67f9147e6e91fd4e04c24246'),
   'data_src': 'mvn_eco',
   'cve_ref': 'CVE-2020-7692',
   'vuln_gav': 'com.google.oauth-client:google-oauth-client:1.23.0',
   'vuln_classes': ['com.google.api.client.auth.oauth2.AuthorizationCodeFlow'],
   'patches': ['https://github.com/googleapis/google-oauth-java-client/commit/13433cd7dd06267fc261f0b1d4764f8e3432c824']}])

In [12]:
import pprint

diff_class_counter = 0

for k in shared_gav_cve_items:
    x = libio_group_by_gav_cve[k]
    if len(x) > 1:
        continue
    x = x[0]
    y = mvn_eco_group_by_gav_cve[k]
    if len(y) > 1:
        continue
    y = y[0]
    
    if set(x['vuln_classes']) == set(y['vuln_classes']):
        continue

    if set(y['vuln_classes']).issubset(set(x['vuln_classes'])):
        continue

    # if set(x['vuln_classes']).issubset(set(y['vuln_classes'])):
    #     continue
        
    diff_class_counter += 1
    pprint.pprint(x)
    print("\n------------\n")
    pprint.pprint(y)
    print("\n@@@@@@@@@@@@@@@\n")

diff_class_counter

{'_id': ObjectId('67f919b35878e3c56f2f1500'),
 'cve_ref': 'CVE-2016-5007',
 'data_src': 'snyk',
 'patches': ['https://github.com/spring-projects/spring-security/commit/e4c13e',
             'https://github.com/spring-projects/spring-framework/commit/a30ab30e4e9ae021fdda04e9abfc228476b846b5'],
 'snyk_url': 'https://security.snyk.io/vuln/SNYK-JAVA-ORGSPRINGFRAMEWORKSECURITY-31336',
 'vuln_classes': ['org.springframework.security.config.annotation.web.AbstractRequestMatcherRegistry',
                  'org.springframework.security.config.annotation.web.builders.HttpSecurity',
                  'org.springframework.security.config.http.MatcherType',
                  'org.springframework.security.config.annotation.web.configurers.UrlAuthorizationConfigurer',
                  'org.springframework.security.config.annotation.web.builders.WebSecurity',
                  'org.springframework.security.config.annotation.web.configurers.ExpressionUrlAuthorizationConfigurer',
                  'or

17

have to remove modified classes that are not in the jar (libio)

In [13]:
mvn_eco_vuln_gav_cve_ref = set((x['vuln_gav'], x['cve_ref']) for x in mvn_eco_vuln_data)
len(mvn_eco_vuln_gav_cve_ref), len(list(x for x in libio_vuln_data if (x['vuln_gav'], x['cve_ref']) not in mvn_eco_vuln_gav_cve_ref))

(829, 5958)

In [14]:
libio_vuln_gav_cve_ref = set((x['vuln_gav'], x['cve_ref']) for x in libio_vuln_data)
len(libio_vuln_gav_cve_ref), len(list(x for x in mvn_eco_vuln_data if (x['vuln_gav'], x['cve_ref']) not in libio_vuln_gav_cve_ref))

(6028, 759)

In [15]:
db.mergedVuln.drop()
db.mergedVuln.insert_many(libio_vuln_data)
_ = db.mergedVuln.insert_many(x for x in mvn_eco_vuln_data if (x['vuln_gav'], x['cve_ref']) not in libio_vuln_gav_cve_ref)

In [16]:
uniq_cve = set()
uniq_cve_ga = set()
uniq_cve_gav_set = set()

for it in db.mergedVuln.find():
    cve_ref = it['cve_ref']
    if cve_ref == '':
        cve_ref = it['snyk_url']
    
    assert cve_ref != '' and cve_ref is not None
    
    uniq_cve.add(cve_ref)
    uniq_cve_ga.add((cve_ref, ':'.join(it['vuln_gav'].split(':')[:-1])))
    uniq_cve_gav_set.add((cve_ref, it['vuln_gav']))

print(len(uniq_cve))  
print(len(uniq_cve_ga))
print(len(uniq_cve_gav_set))

941
977
6787
